# Overlaying the segmented images on the truecolor images


In [5]:
import os
import pandas as pd
import skimage
from skimage.color import label2rgb
import numpy as np
import rasterio as rio
from rasterio.plot import reshape_as_image
from skimage.transform import resize
import proplot as pplt
regions = pd.read_csv('../data/metadata/region_definitions.csv', index_col='region')
cases = pd.read_csv('../data/metadata/validation_dataset_case_list.csv')
cases['start_date'] = pd.to_datetime(cases['start_date'].values)
saveloc = '../data/modis/'
ebseg_loc = '../data/ift_images/ebseg_v0/'
labeled_loc = '../data/validation_images/'

In [28]:
for row, case in cases.iterrows():
    cn = str(case.case_number).zfill(3)
    region = case.region
    start = case.start_date # check start date format
    end = case.start_date + pd.to_timedelta('1d')
    dx = '100km'

    
    for satellite in ['aqua', 'terra']:
        case_folder = '-'.join([cn, region, dx, start.strftime('%Y%m%d'), end.strftime('%Y%m%d')])
        case_folder += '/' + '-'.join([region, start.strftime('%Y-%m-%d'), satellite])
        ebseg_path = ebseg_loc +  '-'.join(
            [cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.tiff'
    
        if os.path.isfile(ebseg_path):

            # Load truecolor image
            with rio.open(
                saveloc + 'truecolor' + '/' + '.'.join(['-'.join([cn, region, dx, start.strftime('%Y%m%d')]),
                                                       satellite, 'truecolor', '250m', 'tiff'])) as truecolor_image:
                tc_imdata = truecolor_image.read()
                tc_bounds = truecolor_image.bounds

            # Load labeled image
            with rio.open(ebseg_loc + '-'.join(
                [cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.tiff') as ebseg_labeled_image:
                ebseg_imdata = resize(ebseg_labeled_image.read().squeeze(), (391, 391), order=0)
                
            left, bottom, right, top = tc_bounds

            # Overlay colored labels on the TC image
            fig, ax = pplt.subplots(width=6)
            if np.any(ebseg_imdata > 0):
                overlay_ebseg = label2rgb(label=ebseg_imdata,
                                           image=reshape_as_image(tc_imdata),
                                           kind='overlay', alpha=0.5,
                                           bg_color=None,
                                           bg_label=0)
                ax.imshow(overlay_ebseg, extent=[left, right, top, bottom])

            else:
                ax.imshow(reshape_as_image(tc_imdata), extent=[left, right, top, bottom])

            # If manually labeled floes exist, overlay an outline
            label_path = labeled_loc + 'labeled_floes_png/' + \
                                           '-'.join([cn, region, start.strftime('%Y%m%d'),
                                                     satellite, 'labeled_floes']) + '.png'
            if os.path.isfile(label_path):
                manually_labeled_image = skimage.io.imread(label_path)[:,:,0]
                eroded_image = skimage.morphology.erosion(manually_labeled_image, skimage.morphology.disk(1))
                borders = np.ma.masked_array(manually_labeled_image, mask=manually_labeled_image - eroded_image == 0) 
                ax.imshow(borders, color='k', alpha=1, extent=[left, right, top, bottom])
                
            ax.axis('off')
            fig.save('../data/ift_images/overlays/' + \
                     '-'.join([cn, region, dx, start.strftime('%Y%m%d'), satellite, 'ebseg_v0']) + '.png',
                     dpi=300, bbox_inches='tight', pad_inches=0)
            pplt.close(fig)
        else:
            print(case.case_number)
       


case_number                                     0
region                                 baffin_bay
start_date                    2004-07-25 00:00:00
center_lon                             -65.289201
center_lat                              69.901354
center_x                                -762500.0
center_y                               -2062500.0
month                                           7
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.486
start_doy                                     207
Name: 0, dtype: object


C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\cor

case_number                                   441
region                                 baffin_bay
start_date                    2020-07-26 00:00:00
center_lon                             -92.095253
center_lat                              76.667722
center_x                               -1062500.0
center_y                                -987500.0
month                                           7
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.789
start_doy                                     208
Name: 15, dtype: object
case_number                                   441
region                                 baffin_bay
start_date                    2020-07-26 00:00:00
center_lon                             -92.095253
center_lat                              76.667722
center_x                               -1062500.0
center_y                                -987500.0
month                                           7
sea_ice_fraction          

C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\cor

case_number                                    82
region                          barents_kara_seas
start_date                    2015-03-03 00:00:00
center_lon                              56.048906
center_lat                              70.318634
center_x                                2112500.0
center_y                                 412500.0
month                                           3
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.818
start_doy                                      62
Name: 35, dtype: object
case_number                                    82
region                          barents_kara_seas
start_date                    2015-03-03 00:00:00
center_lon                              56.048906
center_lat                              70.318634
center_x                                2112500.0
center_y                                 412500.0
month                                           3
sea_ice_fraction          

...

case_number                                   454
region                                 hudson_bay
start_date                    2013-04-26 00:00:00
center_lon                             -76.212147
center_lat                              63.096694
center_x                               -1537500.0
center_y                               -2537500.0
month                                           4
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.865
start_doy                                     116
Name: 141, dtype: object
case_number                                   454
region                                 hudson_bay
start_date                    2013-04-26 00:00:00
center_lon                             -76.212147
center_lat                              63.096694
center_x                               -1537500.0
center_y                               -2537500.0
month                                           4
sea_ice_fraction         

C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)


case_number                                   352
region                                 laptev_sea
start_date                    2007-05-04 00:00:00
center_lon                             137.489553
center_lat                              76.774164
center_x                                 -62500.0
center_y                                1437500.0
month                                           5
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.804
start_doy                                     124
Name: 151, dtype: object
case_number                                   352
region                                 laptev_sea
start_date                    2007-05-04 00:00:00
center_lon                             137.489553
center_lat                              76.774164
center_x                                 -62500.0
center_y                                1437500.0
month                                           5
sea_ice_fraction         

C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)
C:\Users\danie\.conda\envs\cal-val\lib\site-packages\numpy\ma\core.py:467: RuntimeWarning: invalid value encountered in cast
  fill_value = np.array(fill_value, copy=False, dtype=ndtype)


case_number                                   360
region                                 laptev_sea
start_date                    2008-03-30 00:00:00
center_lon                             107.715793
center_lat                              77.710993
center_x                                 612500.0
center_y                                1187500.0
month                                           3
sea_ice_fraction                              1.0
mean_sea_ice_concentration                  0.914
start_doy                                      90
Name: 153, dtype: object
case_number                                   362
region                                 laptev_sea
start_date                    2008-06-01 00:00:00
center_lon                             115.958769
center_lat                                76.2695
center_x                                 487500.0
center_y                                1412500.0
month                                           6
sea_ice_fraction         

In [27]:
manually_labeled_image.max()

255

In [19]:
ebseg_imdata.max()

0

In [89]:
left, bottom, right, top = ebseg_labeled_image.bounds

In [90]:
(right - left)/400

250.0